In [1]:
%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')

from pylab import *

import timeit
sys.path.append('../../')
import rspace
from hamiltonian import Parameters

M_PI = math.pi 

def write_E_imp(fname,ep,tpd,E_imp):
    #"a" - Append - will append to the end of the file
    #"w" - Write - will overwrite any existing content
    f = open('./data_E_imp/'+fname,'a',1) 
    f.write('{:.6e}\t{:.6e}\t{:.6e}\n'.format(ep,tpd,E_imp))
    
def get_ground_state(matrix):       
    print 'start getting ground state'
    # in case eigsh does not work but matrix is actually small, e.g. Mc=1 (CuO4)
    #M_dense = matrix.todense()
    #print 'H='
    #print M_dense
    #vals, vecs = np.linalg.eigh(M_dense)
    #Neval = M_dense.shape[0]
    #print Neval
    
    # in case eigsh works:
    Neval = 15
    vals, vecs = sps.linalg.eigsh(matrix, k=Neval, which='SA')
    print 'lowest eigenvalue of H = '
    print vals, min(vals)
    return min(vals)

def calc_spec_weight_spsolve(wmin,wmax):
    wvals = []
    gf0 = []
    gf1 = []

    rhs = rspace.create_rhs_matrix()
    w = wmin
    dw = prm.eta/4.0
    static_eom = rspace.create_static_eom_matrix(prm)
    
    E_imp = get_ground_state(static_eom)
    
    ''' comment out below for debug later
    static_eom = static_eom.tocsc()          
    static_eom.sort_indices()
    sps.linalg.dsolve.use_solver( useUmfpack = True,\
                                  assumeSortedIndices = True )
    while w < wmax:
        eom = rspace.create_eom_matrix(static_eom,w)
        x = sps.linalg.spsolve(eom.tocsc(),rhs).toarray() 
        gf0.append(x[rspace.get_index(0,0,0),0])
        gf1.append(x[rspace.get_index(0,0,1),1])
        wvals.append(w)
        w += dw
        #print (w-wmin)/(wmax-wmin)

    plt.plot(wvals,-np.imag(gf0)/M_PI,'-b',label='d3z2r2')
    plt.plot(wvals,-np.imag(gf1)/M_PI,'-r',label='dx2y2')
    
    maxval = max(-np.imag(gf0)/M_PI)
    xlim([-5,20])
    ylim([0,maxval])
    xlabel('$\omega$',fontsize=17)
    ylabel('$A(\omega)$',fontsize=17)
    #text(0.45, 0.1, '(a)', fontsize=16)
    grid('on',linestyle="--", linewidth=0.5, color='black', alpha=0.5)
    legend(loc='best', fontsize=9.5, framealpha=1.0, edgecolor='black')
    savefig("Aw.pdf")
    # plt.legend(loc='upper left')
    # plt.show()
    '''
    return E_imp

def calc_spec_weight_bicg(wmin,wmax):
    wvals = []
    gf0 = []
    gf1 = []
    gf2 = []
    rhs0 = rspace.create_rhs_vec(0)
    rhs1 = rspace.create_rhs_vec(1)
    rhs2 = rspace.create_rhs_vec(2)
    w = wmin
    dw = prm.eta/2.0
    static_eom = rspace.create_static_eom_matrix(prm)
    sps.linalg.dsolve.use_solver( useUmfpack = True,\
                                  assumeSortedIndices = True )
    while w < wmax:
        eom = rspace.create_eom_matrix(static_eom,w)
        x0, info = sps.linalg.bicg(eom,rhs0) 
        x1, info = sps.linalg.bicg(eom,rhs1) 
        x2, info = sps.linalg.bicg(eom,rhs2) 
        gf0.append(x0[rspace.get_index(0,0,0)])
        gf1.append(x1[rspace.get_index(0,0,1)])
        gf2.append(x2[rspace.get_index(0,0,2)])
        wvals.append(w)
        w += dw
    #     print (w-wmin)/(wmax-wmin)
    # plt.plot(wvals,-np.imag(gf0)/M_PI,'-r',label='s')
    # plt.plot(wvals,-np.imag(gf1)/M_PI,'-b',label='px')
    # plt.plot(wvals,-np.imag(gf2)/M_PI,'-g',label='py')
    # plt.legend()
    # plt.show()

def calc_spec_weight_pardiso(wmin,wmax):
    wvals = []
    gf0 = []
    gf1 = []
    gf2 = []

    rhs0 = rspace.create_rhs_vec(0)
    rhs1 = rspace.create_rhs_vec(1)
    rhs2 = rspace.create_rhs_vec(2)
    rhs = np.asfortranarray(np.vstack((rhs0,rhs1,rhs2)).T)
    w = wmin
    dw = prm.eta/2.0
    static_eom = rspace.create_static_eom_matrix(prm)
    # static_eom.sort_indices()
    ia = static_eom.indptr+1  # fortran indexing starts at 1
    ja = static_eom.indices+1 # fortran indexing starts at 1
    a = static_eom.data

    pt = np.empty(64, dtype='i8', order='F')
    iparm = np.empty(64, dtype='i4', order='F')
    dparm = np.empty(64, dtype='f8', order='F')
    iparm[2] = 1 # number of processors

    error = pardiso.init(pt, iparm, dparm)
    if error != 0:
        print 'Error in pardiso.init(), ERROR={:d}'.format(error)
    x, error = pardiso.sym_fact(pt,a,ia,ja,iparm,rhs,dparm)
    if error != 0:
        print 'Error in pardiso.sum_fact(), ERROR={:d}'.format(error)
    
    while w < wmax:
        eom = rspace.create_eom_matrix(static_eom,w)
        a = eom.data
        x, error =  pardiso.solve(pt,a,ia,ja,iparm,rhs,dparm)
        if error != 0:
            print 'Error in pardiso.solve(), ERROR={:d}'.format(error)
        gf0.append(x[rspace.get_index(0,0,0),0])
        gf1.append(x[rspace.get_index(0,0,1),1])
        gf2.append(x[rspace.get_index(0,0,2),2])
        wvals.append(w)
        w += dw
    #     print (w-wmin)/(wmax-wmin)
    # plt.plot(wvals,-np.imag(gf0)/M_PI,'-r',label='s')
    # plt.plot(wvals,-np.imag(gf1)/M_PI,'-b',label='px')
    # plt.plot(wvals,-np.imag(gf2)/M_PI,'-g',label='py')
    # plt.legend(loc='upper left')
    # plt.show()

if __name__ == '__main__':   
    Norb = 7
    tpp = 0.55
    ed = 0
    eta = 0.01
    eps = np.arange(-2.15, -2.36, -0.01) #[3.5]#,3.5,4.5]
    
    # or read eps from existing file
    #f1A1 = 'Norb7_tpd0.7_tpp0.55_A6.5_B0.15_C0.58_Upp0_Mc32_eta0.001_1A1.txt'
    #a = loadtxt('../twohole_impurity_model/data_lowpeak/'+f1A1,skiprows=0)
    #eps = a[:,0]
    tpds = [0.6]  #np.arange(0.1, 0.11, 0.1)
    rspace.M = 32
    
    for ep in eps:
        for tpd in tpds:
            print '==================================================='
            print 'ep=', ep, ' tpd=',tpd,' tpp=',tpp
            fEimp = 'E_imp_Norb'+str(Norb)+'_tpd'+str(tpd)+'_tpp'+str(tpp)+'_Mc'+str(rspace.M)+'.txt'

            prm = Parameters(tpd,tpp,ed,ep,eta)
            E_imp = calc_spec_weight_spsolve(-5,20)
            write_E_imp(fEimp,ep,tpd,E_imp)
            # calc_spec_weight_pardiso(-12,2)

Populating the interactive namespace from numpy and matplotlib
ep= -2.15  tpd= 0.6  tpp= 0.55
dim(H) =  41750
start getting ground state
lowest eigenvalue of H = 
[-4.35731059 -4.34708377 -4.34683809 -4.34682956 -4.34494284 -4.34367782
 -4.34178679 -4.34178529 -4.3404715  -4.33925599 -4.33925259 -4.33637083
 -4.33736728 -4.33736727 -4.33422063] -4.357310588842826
ep= -2.1599999999999997  tpd= 0.6  tpp= 0.55
dim(H) =  41750
start getting ground state
lowest eigenvalue of H = 
[-4.36722314 -4.35707757 -4.35682933 -4.35683809 -4.35494284 -4.35367782
 -4.35178529 -4.35178678 -4.35046508 -4.34925599 -4.34925258 -4.34636404
 -4.34736728 -4.34736727 -4.34422063] -4.367223141068575
ep= -2.1699999999999995  tpd= 0.6  tpp= 0.55
dim(H) =  41750
start getting ground state
lowest eigenvalue of H = 
[-4.37713674 -4.36707136 -4.36683809 -4.36682908 -4.36494284 -4.36367782
 -4.36178677 -4.36178529 -4.36045867 -4.35925257 -4.35925599 -4.35635725
 -4.35736727 -4.35736728 -4.35422063] -4.37713673805169
e